In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import json
from tqdm import tqdm_notebook as tqdm
import math
import random
import glob
from tqdm import tqdm_notebook as tqdm

In [3]:
def dump_json(js_meta, name):
    json_data = json.dumps(js_meta)
    f = open(name,"w")
    f.write(json_data)
    f.close()

In [7]:
audio_meta_dir = "metadata_audio_altered.csv"
full_meta_dir = "full_metadata.json"

In [8]:
actually_real_audio = ["yfxtuimubx.mp4","ecumyiowzs.mp4", "ddcfzufmyd.mp4", "axyzsqhnts.mp4", "amrryjxojv.mp4"]

In [9]:
df_audio = pd.read_csv(audio_meta_dir)
print(len(df_audio))
df_audio = df_audio[~df_audio["train_mp4"].isin(actually_real_audio)]
print(len(df_audio))

10157
10152


In [10]:
fake_mp4s = list(df_audio["train_mp4"].values)
list.sort(fake_mp4s)

In [11]:
len(fake_mp4s)

10152

In [13]:
dict_meta =json.load(open(full_meta_dir,'rb'))

In [14]:
len(dict_meta)

119146

In [15]:
wavs = list(glob.iglob("../dltraining/wavs/*.wav"))
wavs = [x.replace("../dltraining/wavs/","").replace(".wav",".mp4") for x in wavs]

In [16]:
no_audio_meta_item = []
for key, v in tqdm(dict_meta.items()):
    if key not in wavs:
        print(key, v)
        no_audio_meta_item.append(key)

taqnnsyxip.mp4 {'label': 'FAKE', 'split': 'train', 'original': 'pjibpowymk.mp4', 'data_dir': '/dltraining/datasets/data/dfdc_train_part_5', 'folder_name': 'dfdc_train_part_5'}
teehidqtii.mp4 {'label': 'REAL', 'split': 'train', 'data_dir': '/dltraining/datasets/data/dfdc_train_part_5', 'folder_name': 'dfdc_train_part_5'}
pjibpowymk.mp4 {'label': 'REAL', 'split': 'train', 'data_dir': '/dltraining/datasets/data/dfdc_train_part_5', 'folder_name': 'dfdc_train_part_5'}
inkxxqwrzi.mp4 {'label': 'FAKE', 'split': 'train', 'original': 'teehidqtii.mp4', 'data_dir': '/dltraining/datasets/data/dfdc_train_part_5', 'folder_name': 'dfdc_train_part_5'}
clmgkufday.mp4 {'label': 'FAKE', 'split': 'train', 'original': 'uyogufqlec.mp4', 'data_dir': '/dltraining/datasets/data/dfdc_train_part_15', 'folder_name': 'dfdc_train_part_15'}
uyogufqlec.mp4 {'label': 'REAL', 'split': 'train', 'data_dir': '/dltraining/datasets/data/dfdc_train_part_15', 'folder_name': 'dfdc_train_part_15'}
inluuhjteb.mp4 {'label': 'REAL

In [17]:
no_audio_meta_item

['taqnnsyxip.mp4',
 'teehidqtii.mp4',
 'pjibpowymk.mp4',
 'inkxxqwrzi.mp4',
 'clmgkufday.mp4',
 'uyogufqlec.mp4',
 'inluuhjteb.mp4',
 'qawhqaraxg.mp4',
 'afnbrsikom.mp4',
 'dyxnxclbhg.mp4',
 'xqnvbrujjo.mp4',
 'strqdkvswm.mp4',
 'cozgtibuda.mp4',
 'qqtoccrfzs.mp4',
 'ihqurgivsl.mp4',
 'szkoxlypql.mp4',
 'ggjareyaqd.mp4',
 'ykpvzfdqpw.mp4',
 'urnqalzaon.mp4',
 'hycdczdeby.mp4',
 'wfdlblolif.mp4',
 'ctpexqamtx.mp4',
 'rbvqghnbez.mp4',
 'xsrbccytqi.mp4',
 'zilqngcakp.mp4',
 'vebpwqhssp.mp4',
 'sxqroedlhr.mp4',
 'wnqlzimgbg.mp4',
 'rjudlivnao.mp4',
 'rsvtkzxufe.mp4']

In [18]:
for nami in no_audio_meta_item:
    del dict_meta[nami]

In [19]:
len(dict_meta)

119116

In [23]:
for key, value in tqdm(dict_meta.items()):
    if key in fake_mp4s:
        value['audio_label'] = 'FAKE' 
        dict_meta[key] = value
    else:
        value['audio_label'] = 'REAL' 
        dict_meta[key] = value

In [21]:
dump_json(dict_meta, "full_audio_meta2.json")

In [29]:
df_meta = pd.DataFrame(dict_meta).T

In [33]:
folders_with_fake_audio = list(df_meta[df_meta["audio_label"]=="FAKE"]["folder_name"].unique())

In [36]:
fake_sample_training_folder = random.sample(folders_with_fake_audio, int(len(folders_with_fake_audio)*0.8))

In [43]:
fake_sample_valid_folder = list(set(folders_with_fake_audio)-set(fake_sample_training_folder))

In [49]:
df_valid = df_meta[df_meta["folder_name"].isin(fake_sample_valid_folder)]
df_train = df_meta[df_meta["folder_name"].isin(fake_sample_training_folder)]

In [52]:
df_train[df_train["audio_label"]=="FAKE"]

,label,split,original,data_dir,folder_name,audio_label
nunyzzmfdw.mp4,FAKE,train,lbgnipxpnx.mp4,/dltraining/datasets/data/dfdc_train_part_4,dfdc_train_part_4,FAKE
osshfmdpgr.mp4,FAKE,train,avpxuvplbn.mp4,/dltraining/datasets/data/dfdc_train_part_4,dfdc_train_part_4,FAKE
ziphbixywq.mp4,FAKE,train,lbgnipxpnx.mp4,/dltraining/datasets/data/dfdc_train_part_4,dfdc_train_part_4,FAKE
iuoaacarhc.mp4,FAKE,train,avpxuvplbn.mp4,/dltraining/datasets/data/dfdc_train_part_4,dfdc_train_part_4,FAKE
qsawsatkfg.mp4,FAKE,train,zveogsfnoz.mp4,/dltraining/datasets/data/dfdc_train_part_4,dfdc_train_part_4,FAKE
...,...,...,...,...,...,...
tvxrrbkzog.mp4,FAKE,train,wgswphznis.mp4,/dltraining/datasets/data/dfdc_train_part_49,dfdc_train_part_49,FAKE
achekamhkd.mp4,FAKE,train,yyfknjxaey.mp4,/dltraining/datasets/data/dfdc_train_part_49,dfdc_train_part_49,FAKE
ozjvlnehul.mp4,FAKE,train,cywebjaezn.mp4,/dltraining/datasets/data/dfdc_train_part_49,dfdc_train_part_49,FAKE
tnaeehpcnb.mp4,FAKE,train,moyljhcxok.mp4,/dltraining/datasets/data/dfdc_train_part_49,dfdc_train_part_49,FAKE


In [53]:
df_valid[df_valid["audio_label"]=="FAKE"]

,label,split,original,data_dir,folder_name,audio_label
ohaqlzfnuv.mp4,FAKE,train,sttnfyptum.mp4,/dltraining/datasets/data/dfdc_train_part_0,dfdc_train_part_0,FAKE
kmcdjxmnoa.mp4,FAKE,train,sttnfyptum.mp4,/dltraining/datasets/data/dfdc_train_part_0,dfdc_train_part_0,FAKE
ihnhxathkq.mp4,FAKE,train,sttnfyptum.mp4,/dltraining/datasets/data/dfdc_train_part_0,dfdc_train_part_0,FAKE
hsxrmrcaqh.mp4,FAKE,train,upmgtackuf.mp4,/dltraining/datasets/data/dfdc_train_part_0,dfdc_train_part_0,FAKE
btizoxwqwp.mp4,FAKE,train,fnslimfagb.mp4,/dltraining/datasets/data/dfdc_train_part_0,dfdc_train_part_0,FAKE
...,...,...,...,...,...,...
taprhjnthk.mp4,FAKE,train,hgmpsbvzkx.mp4,/dltraining/datasets/data/dfdc_train_part_46,dfdc_train_part_46,FAKE
ejpftkbwln.mp4,FAKE,train,irxfiqsbuj.mp4,/dltraining/datasets/data/dfdc_train_part_46,dfdc_train_part_46,FAKE
dhmovshgex.mp4,FAKE,train,tkxblcleyd.mp4,/dltraining/datasets/data/dfdc_train_part_46,dfdc_train_part_46,FAKE
tglxshkgjx.mp4,FAKE,train,qepouweoaa.mp4,/dltraining/datasets/data/dfdc_train_part_46,dfdc_train_part_46,FAKE


In [56]:
len(df_train), len(df_valid)

(97390, 21726)

In [64]:
dict_train_meta = df_train.to_dict(orient='index')
dict_valid_meta = df_valid.to_dict(orient='index')

In [65]:
dump_json(dict_train_meta, "train_audio_meta2.json")
dump_json(dict_valid_meta, "valid_audio_meta2.json")

## old method of splitting data, huge dataleak

In [17]:
train_num = math.ceil(len(fake_mp4s)*0.8)

In [18]:
training_fake_mp4s = fake_mp4s[0:train_num]
valid_fake_mp4s = fake_mp4s[train_num:]
len(training_fake_mp4s), len(valid_fake_mp4s)

(8122, 2030)

In [19]:
all_meta_mp4s = list(dict_meta.keys())

In [20]:
real_mp4s = list(set(all_meta_mp4s)-set(training_fake_mp4s)-set(valid_fake_mp4s))

In [21]:
num_valid = len(valid_fake_mp4s)
valid_real_mp4s = random.sample(real_mp4s,num_valid)
train_real_mp4s = list(set(real_mp4s)-set(valid_real_mp4s))

In [22]:
valid_mp4s = valid_real_mp4s + valid_fake_mp4s
train_mp4s = train_real_mp4s + training_fake_mp4s

In [23]:
len(valid_mp4s),len(train_mp4s)

(4060, 115056)

In [24]:
dict_train_meta = {}
for key, value in tqdm(dict_meta.items()):
    if key in train_mp4s:
        dict_train_meta[key] = value

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [25]:
dict_valid_meta = {}
for key, value in tqdm(dict_meta.items()):
    if key in valid_mp4s:
        dict_valid_meta[key] = value

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [26]:
len(dict_train_meta), len(dict_valid_meta)

(115056, 4060)

In [27]:
dump_json(dict_train_meta, "train_audio_meta2.json")
dump_json(dict_valid_meta, "valid_audio_meta2.json")